In [ ]:
import pandas as pd
import numpy as np

# Summarizing, Aggregating & Grouping
Knowing how to use pandas aggregation and grouping functions lets us look at our data in a different way and quickly find more specific insights.

In [ ]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)

Answer to exercise from notebook 1:


`scrambled_wine[['points', 'region_1']].iloc[:5] 
`

## Initial quick analysis using pandas
Pandas has multiple built-in functions that make it easy to quickly see what's in your dataframe. 
You can build them with the selecting tools you used before.

Here, we will select a column, and then see how pandas lets us quickly analyse it.

To quickly see which columns our wine dataset has, we can use the .columns attribute.

In [ ]:
wine.columns

Let's select price.

In [ ]:
wine['price'].head()

We can quickly see some metrics of the price, using some built-in aggergating functions in pandas.

In [ ]:
print('Average wine price: ', wine['price'].mean())
print('Min wine price: ', wine['price'].min())
print('Median wine price: ', wine['price'].median())
print('Max wine price: ', wine['price'].max())

And some more advanced metrics..

In [ ]:
wine['country'].value_counts()

What if we want to see this list in the opposite order?

In [ ]:
wine['country'].value_counts().sort_values(ascending=True)

Both of these lists are too long. What if we only want to see the top 10 countries?
We can string together the other selectors we learned before!

Can you think of another way to get the top 10 rows?

In [ ]:
wine['country'].value_counts().head(10)

What if we just want to know how many countries are on the list?

In [ ]:
# Number of non-null unique values
wine['country'].nunique()

And if we want a list of them? 
(This isn't a pandas thing, but is still super useful:)

In [ ]:
# Gives all unique values
set(wine['country'])

You can look [here](https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/) for a list of all the built-in pandas stats.

One of the most powerful built-in summary tools for pandas is `df.describe()`. This quickly calculates some of these stats for the numeric columns in the df.

In [ ]:
wine.describe()

### Conditional Selections 
We can use conditional selections to narrow our analysis even further.

DON'T FORGET - to make things easier, we can save selections we plan to use often as their own variables.

In [ ]:
small_index = wine[wine.index<10]
small_index

In [ ]:
us = wine[wine['country']=='US']
france = wine[wine['country']=='France']

In [ ]:
print('Mean American wine price: $', round(us['price'].mean(),2))
print('Mean French wine price: $', round(france['price'].mean(),2))
print('Mean overall wine price is: $', round(wine['price'].mean(),2))

We can then use these to calculate more targeted metrics.

In [ ]:
print('French wine is ${} more expensive on average'.format(round(france['price'].mean() - wine['price'].mean(),2)))

#### More advanced conditionals: Using masks
When you want to filter on >1 criteria, it can be easier to use a mask.

How many wines from North America do we have on our list?

In [ ]:
na_mask = (wine.country == 'US') | (wine.country == 'Mexico') | (wine.country == 'Canada') 
na_indexes = wine.index[na_mask]
na = wine.loc[na_indexes,:]

In [ ]:
wine[na_mask]

In [ ]:
na.head()

How many wines do we have total in the in North America?

In [ ]:
na['country'].count()

**Question:** How many of the wines belong to each country?

**Question:** From which US state do most of our wines come?

In [ ]:
na['country'].value_counts()

In [ ]:
wine[wine['country']=='US']['province'].value_counts().index[0]

In [ ]:
wine[wine['country'] == 'US'].groupby('province').count().sort_values(by='country', ascending=False)['country'].index[0]

## Groupby
One of the most flexible ways to aggregate in pandas is with .groupby() .
We will look at how this works for categorical datasets like this one, and also for datetime datasets, as dealing with datetimes in pandas can be tricky.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### How Groupby Works:
You can group your data in many different ways, and also aggregate it by any of the aggregators we saw before: like mean, mode, sum, etc.

In [ ]:
wine.groupby(['country']).mean().sort_values(by=['points','price'], ascending=[False,True])

So, British wine is the highest average ranked?? This is surprising. Maybe we need to look at our data in a different way.

The beauty of `df.groupby()` is that it lets you aggregate different columns in different ways.

Say we want to know the average price of wine in each country, but the _highest_ score:

In [ ]:
wine.groupby('country').agg({'price':np.mean, 'points':'max'}).sort_values(by=['points','price'], ascending=[False, True]).reset_index().head(10)

**Tip:** Sometimes, fo make your code cleaner it's best to move the aggregations out and store them in their own variable that you can update saparately.

In [ ]:
aggs = {
    'price': 'mean',
    'points': 'max'
}

wine.groupby('country').agg(aggs).sort_values(by=['points','price'], ascending =[False, True]).head()

You can also use a list in your aggs to aggergate one column in different ways.

This will give a **multi-index**. Multi-indexes can be difficult to sort on. But, there are a few different ways we can deal with this.

In [ ]:
aggs = {
    'price': ['min', 'mean', 'max', 'std']
}

price_table = wine.groupby('country').agg(aggs)
price_table

One way is by dropping the top level ('price'):

In [ ]:
price_table.columns = price_table.columns.droplevel(level=0)

# now, we can sort by any of the columns. Here, by average price.
price_table.sort_values(by='mean', ascending=False).head()

Another way is by using `np.ravel()` . This preserves the "price" indicator somewhere in each of the column names.

In [ ]:
price_table = wine.groupby('country').agg(aggs)

# Using ravel, and a string join, we can create better names for the columns:
price_table.columns = ["_".join(x) for x in price_table.columns.ravel()]
price_table.sort_values(by='price_mean', ascending=False).head()

**Question**: Create a quick plot of the number of wines made in each country, from highest to lowest.

In [ ]:
wine.groupby('country')['price'].count().sort_values(ascending=False)

In [ ]:
wine['country'].value_counts().plot(kind='bar', figsize=(10,7))
plt.show()

# Dealing with datetime in Pandas
Pandas built off the datetime package in Python to offer a datetime index, and plenty of ways to work with this.
However, it is still far from intuitive. 
That doesn't mean it's not useful, and for anyone doing a timeseries project, you'll need to deal with dates and times in pandas often.

Let's load a sample dataset of datetime energy data and get started!

In [ ]:
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

Python Datetime functionality example:

In [ ]:
from datetime import datetime

s = "8 March, 2017"
d = datetime.strptime(s, '%d %B, %Y')
print(d.strftime('%Y-%m-%d'))

In [ ]:
d

Note that this data has a DateTimeIndex. 
setting `parse_dates=True` when we read the CSV lets pandas infer this datetimeindex.

In [ ]:
energy.index

We can select data points within a specific time range, using the DateTimeIndex and .loc.
Here, we select the first one day of data.

In [ ]:
energy.loc['1998-12-31 01:00:00':'1999-01-01 00:00:00', :]

In [ ]:
# One record for each hour of this day.
energy.loc['1998-12-31 01:00:00':'1999-01-01 00:00:00', :].shape[0]

## Selecting with boolean indexing on pandas datetimeindex
We can use .dot notation with conditionals to select on specific parts of the datetime, like days or months.

In [ ]:
# making a new DF that only includes the month of septmeber from each year.
septembers = energy[energy.index.month == 9]

We can also call just a date, and get all the hours/time periods in that day:

In [ ]:
energy['2000-01-03']

Same with for a year and month:

In [ ]:
month_jan = energy['2000-01'].shape

In [ ]:
# We see that it includes one record for each our of each day of the month of January, which has 31 days
assert month_jan[0] == 31*24

## Resampling
We can also combine the data in different ways, and over different time periods.
This means that just because our data is in hourly time periods, we dont have to keep it that way. 

In [ ]:
# We can get the average load over a day:
daily_avg_energy = energy.resample('D').mean()
daily_avg_energy.head()

In [ ]:
# We can also get the total MWh used in a day:
daily_energy = energy.resample('D').sum()
daily_energy.columns = ["PJM_Load_MWh"]
daily_energy.head()

## Groupby with DateTimeIndex
Using groupby with a pandas DateTimeIndex can be extremely useful and powerful.
Let's look at how this can work.

In [ ]:
index_level_names = ['year', 'month', 'week', 'day']
indexes = [energy.index.year.rename('year'),
           energy.index.month.rename('month'),
           energy.index.week.rename('week'),
           energy.index.day.rename('day')]

aggregated = energy.groupby(indexes).sum()

In [ ]:
aggregated

In [ ]:
aggregated.head()

The index level names are not particularly helpful here. We can change them.

### Selecting on Multi-Index: Using Slice

In [ ]:
energy_2000 = aggregated.loc[(slice(2000,2000), slice(None), slice(None), slice(None)), :]
energy_2000

Or, say we want to know what Christmas week looked like across all the years for which we have data..

In [ ]:
energy_christmas = aggregated.loc[(slice(None), slice(None), slice(52,52), slice(None)), :]

You can also then use a groupby again!

Note that here, we're using a groupby on the index level.

In [ ]:
energy_christmas.groupby(level='year').sum()

Now, we can plot by these levels!

In [ ]:
energy_christmas.groupby(level=0).sum().plot(kind='bar')
plt.show()

**Question:** See what just writing "52" in the slice box does.

In [ ]:
# Gets all weeks up to and including the 52nd week.
aggregated.loc[(slice(None), slice(None), slice(52), slice(None)), :]

**Question**: Select Christmas week, but only for years 1998-2000.

In [ ]:
christmas_98_00 = aggregated.loc[(slice(1998, 2000), slice(None), slice(52,52), slice(None)), :]
christmas_98_00

**Question**: Which week is christmas in all of the years?

In [ ]:
christmas_day = aggregated.loc[(slice(None), slice(12,12), slice(None), slice(24,24)), :]
christmas_day.index.get_level_values(2)

## Exercise:
- Find the week (and its associated year) with the highest total weekly consumption.

- Find the day of the week that averages the highest consumption

- Find the time of day that averages the lowest consumption.
    - Has this changed over the years?
    

- Is average consumption rising, falling, or staying the same over the years?
- What is the %age difference in consumption on average between April and June?

Find the week (and its associated year) with the highest total weekly consumption.

In [ ]:
weekly = energy.resample('W').sum()

In [ ]:
energy.resample('W').sum().sort_values(by='PJM_Load_MW', ascending=False).index[0]

In [ ]:
weekly.idxmax()

In [ ]:
weekly.loc['2001-08-12']

In [ ]:
energy.groupby([energy.index.year, energy.index.week]).sum().sort_values(by='PJM_Load_MW', ascending=False).index[0]

Find the day of the week that averages the highest consumption

In [ ]:
day_of_week = energy.groupby(energy.index.weekday).mean()
day_of_week.sort_values(by='PJM_Load_MW', ascending=False).index[0]

# 1 = Tuesday.

In [ ]:
day_of_week

Find the time of day that averages the lowest consumption.
Has this changed over the years?

In [ ]:
time_of_day = energy.groupby(energy.index.hour).mean()
time_of_day.sort_values(by='PJM_Load_MW', ascending=True).index[0]

In [ ]:
time_of_day = energy.groupby([energy.index.year, energy.index.hour]).mean()
time_of_day.groupby(level=0).idxmin()
# In the last year it changed to be midnight 

Is average consumption rising, falling, or staying the same over the years?

In [ ]:
energy.groupby(energy.index.year).mean()
# it's increasing

In [ ]:
avg_monthly_consumption

What is the %age difference in consumption on average between April and June?

In [ ]:
avg_monthly_consumption = energy.groupby(energy.index.month).mean()
april_consump = avg_monthly_consumption.loc[4]
june_consump = avg_monthly_consumption.loc[6]

percent_diff = ((june_consump - april_consump) / april_consump ) * 100

print('April consumption average is:', int(april_consump))
print('June consumption average is:', int(june_consump))
print('Percentage Difference in consumption average is:', int(percent_diff))

In [ ]:
#df.groupby([df.Date.dt.year.rename('year'), df.Date.dt.month.rename('month')]).mean()